# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-217987
Azure region: southcentralus
Subscription id: 3e42d11f-d64d-4173-af9b-12ecaa1030b3
Resource group: aml-quickstarts-217987


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "auto-ml"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2',                                      
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)
# For a more detailed view of current AmlCompute status, use get_status().
compute_target.wait_for_completion(show_output=True)


Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [7]:
from azureml.core.dataset import Dataset

found = False
key = "heart_failure_clinical_records"
description_text = "Heart Failure Prediction"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        my_dataset = 'https://raw.githubusercontent.com/jeeva-jose/Capstone-Project/main/heart_failure_clinical_records_dataset.csv'
        dataset = Dataset.Tabular.from_delimited_files(my_dataset)        
        # Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)

df = dataset.to_pandas_dataframe()
df.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [ ]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = <your policy here>

#TODO: Create the different params that you will be using during training
param_sampling = <your params here>

#TODO: Create your estimator and hyperdrive config
estimator = <your estimator here>

hyperdrive_run_config = <your config here>

In [8]:

from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler

ps = RandomParameterSampling({
    '--C': choice(1.0, 0.1, 0.05),
    '--max_iter': choice(50,100,150)})

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
#src = ### YOUR CODE HERE ###
training_folder ='./training'
src = ScriptRunConfig(source_directory=training_folder,
                      script='train.py',
                      compute_target=compute_target,
                      environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
 ### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(run_config=src,
                             hyperparameter_sampling=ps,
                             policy=policy,
                             primary_metric_name="Accuracy",
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs=50,
                             max_concurrent_runs=4)

In [9]:
#TODO: Submit your experiment
hyperdrive_run = exp.submit(hyperdrive_config, show_output = True)


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [10]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_763e0c1e-1f89-4003-b709-78ba250f310b
Web View: https://ml.azure.com/runs/HD_763e0c1e-1f89-4003-b709-78ba250f310b?wsid=/subscriptions/3e42d11f-d64d-4173-af9b-12ecaa1030b3/resourcegroups/aml-quickstarts-217987/workspaces/quick-starts-ws-217987&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2022-12-10T10:14:12.865837][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2022-12-10T10:14:14.1815204Z][SCHEDULER][INFO]Scheduling job, id='HD_763e0c1e-1f89-4003-b709-78ba250f310b_0' 
[2022-12-10T10:14:14.3600225Z][SCHEDULER][INFO]Scheduling job, id='HD_763e0c1e-1f89-4003-b709-78ba250f310b_2' 
[2022-12-10T10:14:14.3011033Z][SCHEDULER][INFO]Scheduling job, id='HD_763e0c1e-1f89-4003-b709-78ba250f310b_1' 
[2022-12-10T10:14:14.4772445Z][SCHEDULER][INFO]Scheduling job, id='HD_763e0c1e-1f89-4003-b709-78ba250f310b_3' 
[2022-12-10T10:14:14.439301][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution t

{'runId': 'HD_763e0c1e-1f89-4003-b709-78ba250f310b',
 'target': 'auto-ml',
 'status': 'Completed',
 'startTimeUtc': '2022-12-10T10:14:12.019867Z',
 'endTimeUtc': '2022-12-10T10:32:44.852763Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'bf504522-4201-4957-b36d-49eedecc0885',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1022-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.47.0',
  'space_size': '9',
  'score': '0.8',
  'best_child_run_id': 'HD_763e0c1e-1f89-4003-b709-78ba250f310b_1',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_763e0c1e-1f89-4003-b709-78ba250f310b_1'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClientType': 'azureml-sdk-t

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [11]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()

# get_metrics()
# Returns the metrics from all the runs that were launched by this HyperDriveRun.
print("Best run metrics :",best_run.get_metrics())
print('===================================================')

# get_details()
# Returns a dictionary with the details for the run
print("Best run details :",best_run.get_details())
print('===================================================')

# get_file_names()
# Returns a list of the files that are stored in association with the run.

print("Best run file names :",best_run.get_file_names())
print('===================================================')

Best run metrics : {'Regularization Strength:': 1.0, 'Max iterations:': 150, 'Accuracy': 0.8}
Best run details : {'runId': 'HD_763e0c1e-1f89-4003-b709-78ba250f310b_1', 'target': 'auto-ml', 'status': 'Completed', 'startTimeUtc': '2022-12-10T10:27:20.485269Z', 'endTimeUtc': '2022-12-10T10:28:19.250499Z', 'services': {}, 'properties': {'_azureml.ComputeTargetType': 'amlctrain', 'ContentSnapshotId': 'bf504522-4201-4957-b36d-49eedecc0885', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFile': 'azureml-logs/process_status.json'}, 'inputDatasets': [], 'outputDatasets': [], 'runDefinition': {'script': 'train.py', 'command': '', 'useAbsolutePath': False, 'arguments': ['--C', '1', '--max_iter', '150'], 'sourceDirectoryDataStore': None, 'framework': 'Python', 'communicator': 'None', 'target': 'auto-ml', 'dataReferences': {}, 'data': {}, 'outputData': {}, 'datacaches': [], 'jobName': None, 'maxRunDurationSeconds': 2592000, 'nodeCount': 1, 'instanceTypes': [], 'priority': None, 

In [12]:
#TODO: Save the best model
best_run.register_model(model_name = "best_run_hyperdrive.pkl", model_path = './outputs/')
print(best_run)

Run(Experiment: udacity-project,
Id: HD_763e0c1e-1f89-4003-b709-78ba250f310b_1,
Type: azureml.scriptrun,
Status: Completed)


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

